# Prepare DoS Hping Attack Dataset

## Overview:

This notebook will focus on creating a DoS Hping attack dataset based on a small sample of data collected by performing real DoS TCP SYN Flood attacks in a controlled environment.<br>
The dataset that this notebook creates closely represents real-world data and was used to train our SVM model.<br>  
There are multiple sample datasets because we performed the attack in a few different ways, and in each way, the data is slightly different.<br>
That is why we split the original sample dataset into multiple samples, ensuring that the attack dataset we generate matches the real-world data as closely as possible.<br>  
It is worth noteing that the sample dataset we collected does not contain any missing values or any outliers due to the fact we tested each part of the collection process and verified that it is correct.<br>
In this notebook we have generated an attack dataset with 11,500 flows of the DoS Hping attack based on the samples we collected when running a DoS TCP SYN Flood attacks in various configurations using the well known DoS Hping3 tool.<br> 

## Imports & Global Variables:

In [1]:
import pandas as pd
import numpy as np
import random

NUM_OF_ROWS = 11500
ATTACK_NAME = 'DoS'

In [2]:
# the following command will make it so that when we print the dataframe we will see all the columns
pd.set_option('display.max_columns', None)

---

## Load the first sample dataset:

In [3]:
# import the attack sample dataset
dos_samples = pd.read_csv('dos_hping_samples_1.csv')
print(f'Dataset Shape: {dos_samples.shape}')
dos_samples

Dataset Shape: (18, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,259948,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,57.282503,9998,0,0,2.773013,3605.464376,1.589317,0.000277,0.019216
1,1,60.0,60,60,0.0,0.0,259922,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,31.935373,9997,0,0,4.922741,2030.779220,1.713713,0.000492,0.028124
2,1,60.0,60,60,0.0,0.0,259974,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,30.416988,9999,0,0,5.267316,1898.310308,1.850942,0.000527,0.030091
3,1,60.0,60,60,0.0,0.0,236886,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.008564,9111,0,0,44.188204,206.186248,44.076651,0.004851,0.461770
4,1,60.0,60,60,0.0,0.0,256802,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.036906,9877,0,0,161.398189,61.196473,161.285302,0.016342,1.622864
5,1,60.0,60,60,0.0,0.0,259922,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,60.236848,9997,0,0,2.909270,3436.257430,1.548943,0.000291,0.019841
6,1,60.0,60,60,0.0,0.0,256568,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.013181,9868,0,0,81.048627,121.754068,80.927725,0.008214,0.814672
7,1,60.0,60,60,0.0,0.0,259974,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,64.413776,9999,0,0,2.683720,3725.798694,1.323679,0.000268,0.018444
8,1,60.0,60,60,0.0,0.0,259610,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.015633,9985,0,0,101.352469,98.517580,101.220045,0.010151,1.012960
9,1,60.0,60,60,0.0,0.0,259792,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,26.013017,9992,0,0,45.536350,219.429093,45.409360,0.004558,0.454275


### Find the columns that we need to synthesis data for:

In [4]:
columns_to_gather = dos_samples.replace(0, np.nan) #replace all 0 values with null
columns_to_gather = columns_to_gather.dropna(how = 'all', axis = 1).columns.tolist() #remove all columns where there are null values
columns_to_gather #left with all columns that the values are not 0 (we know for a fact that the data is consistant and there are not missing values in the rows we to the collection process)

['Number of Ports',
 'Average Packet Length',
 'Packet Length Min',
 'Packet Length Max',
 'Total Length of Fwd Packet',
 'Fwd Packet Length Max',
 'Fwd Packet Length Mean',
 'Fwd Packet Length Min',
 'Fwd Segment Size Avg',
 'Subflow Fwd Bytes',
 'SYN Flag Count',
 'Flow Duration',
 'Packets Per Second',
 'IAT Max',
 'IAT Mean',
 'IAT Std']

### Find an approximate minimum and maximum values of each column:

In [5]:
# find the minimum and maximum values for each column, scale the range (reduce min by 15% and increase max by 10%), and store the results in a dictionary.
min_max_dict = {col: (dos_samples[col].min() * 0.85, dos_samples[col].max() * 1.1) for col in columns_to_gather}
min_max_dict['Number of Ports'] = (1, 1) #ensure that the 'Number of Ports' column always has the value '1'

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 1.00 | Max: 1.00
Average Packet Length          | Min: 51.00 | Max: 66.00
Packet Length Min              | Min: 51.00 | Max: 66.00
Packet Length Max              | Min: 51.00 | Max: 66.00
Total Length of Fwd Packet     | Min: 59891.00 | Max: 286000.00
Fwd Packet Length Max          | Min: 22.10 | Max: 28.60
Fwd Packet Length Mean         | Min: 22.10 | Max: 28.60
Fwd Packet Length Min          | Min: 22.10 | Max: 28.60
Fwd Segment Size Avg           | Min: 5.10 | Max: 6.60
Subflow Fwd Bytes              | Min: 22.11 | Max: 494.81
SYN Flag Count                 | Min: 2303.50 | Max: 11000.00
Flow Duration                  | Min: 1.34 | Max: 177.54
Packets Per Second             | Min: 25.74 | Max: 6988.15
IAT Max                        | Min: 1.13 | Max: 177.41
IAT Mean                       | Min: 0.00 | Max: 0.04
IAT Std                        | Min: 0.01 | Max: 2.20


### Create the base attack dataset (full of zeros):

In [6]:
# creating an empty dataframe before adding values to it
dos_dataset = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dos_samples.columns))), columns = dos_samples.columns)
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [7]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dos_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dos_dataset[col] = int(0)
zero_columns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

In [8]:
dos_dataset.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0


---

## Filling in values based on collected samples:

### Firstly we insert data into columns that have the exact same values:

For each one of the 'same_value' columns we insert the same vector, meaning that in each row of the attack dataset, these columns will have the same value.

In [9]:
same_value = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max'] #based on collected samples
val = np.random.randint(min_max_dict[same_value[0]][0], min_max_dict[same_value[0]][1]*1.1, NUM_OF_ROWS)

for col in same_value:
    dos_dataset[col] = val

In [10]:
same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min'] #based on collected samples
val2 = np.random.randint(min_max_dict[same_value2[0]][0], min_max_dict[same_value2[0]][1]*1.25, NUM_OF_ROWS)

for col in same_value2:
    dos_dataset[col] = val2

In [11]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,68,68,68,0,0,0.0,32,32,32,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
1,0.0,64,64,64,0,0,0.0,28,28,28,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
2,0.0,66,66,66,0,0,0.0,34,34,34,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
3,0.0,51,51,51,0,0,0.0,33,33,33,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
4,0.0,55,55,55,0,0,0.0,33,33,33,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,0.0,57,57,57,0,0,0.0,33,33,33,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
11496,0.0,60,60,60,0,0,0.0,28,28,28,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
11497,0.0,64,64,64,0,0,0.0,34,34,34,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
11498,0.0,68,68,68,0,0,0.0,34,34,34,0,0,0,0,0,0.0,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0


In [12]:
dos_dataset['Fwd Segment Size Avg'] = np.random.randint(min_max_dict['Fwd Segment Size Avg'][0]*0.9, min_max_dict['Fwd Segment Size Avg'][1]*1.5, NUM_OF_ROWS)
dos_dataset['Number of Ports'] = np.full(shape = NUM_OF_ROWS, fill_value = 1, dtype = int)

Some columns, like 'SYN Flag Count', based on the collected samples, usually have values in a specific range, but sometimes they have values outside of the range.<br>
In order to generate accurate data, we generate a vector that will have a certain distribution of values. For example, in the 'SYN Flag Count' column, 90% of the values will be within the usual range,<br>
but the other 10% will have values that are anywhere between the minimal and maximal value for this column, meaning they will have values outside of the usual range as well.  

In [13]:
rand_values = dos_dataset['SYN Flag Count'] = np.random.randint(min_max_dict['SYN Flag Count'][0], min_max_dict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)
usual_values = np.random.randint(8176, 10658, NUM_OF_ROWS)

# choose values randomly (10% from rand_values, 90% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.1, usual_values, rand_values) 

dos_dataset['SYN Flag Count'] = chosen_values 

In [14]:
rand_values = np.random.uniform(min_max_dict['Flow Duration'][0], min_max_dict['Flow Duration'][1], NUM_OF_ROWS)
usual_values = np.random.uniform(1.654, 45.175, NUM_OF_ROWS)

# choose values randomly (25% from rand_values, 75% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.25, usual_values, rand_values) 

dos_dataset['Flow Duration'] = chosen_values

In [15]:
rand_values = np.random.uniform(min_max_dict['Subflow Fwd Bytes'][0], min_max_dict['Subflow Fwd Bytes'][1], NUM_OF_ROWS)
usual_values = np.random.uniform(13.763, 72.146, NUM_OF_ROWS)

# choose values randomly (10% from rand_values, 90% from usual_values)
chosen_values = np.where(np.random.rand(NUM_OF_ROWS) > 0.1, usual_values, rand_values) 

dos_dataset['Subflow Fwd Bytes'] = chosen_values

In [16]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,68,68,68,0,0,0.0,32,32,32,0,0,0,0,0,7,0,25.180855,9806,0,0,42.414232,0.0,0.0,0.0,0.0
1,1,64,64,64,0,0,0.0,28,28,28,0,0,0,0,0,5,0,42.078845,8586,0,0,133.656006,0.0,0.0,0.0,0.0
2,1,66,66,66,0,0,0.0,34,34,34,0,0,0,0,0,4,0,60.819446,10630,0,0,18.534921,0.0,0.0,0.0,0.0
3,1,51,51,51,0,0,0.0,33,33,33,0,0,0,0,0,5,0,48.427463,9355,0,0,33.887945,0.0,0.0,0.0,0.0
4,1,55,55,55,0,0,0.0,33,33,33,0,0,0,0,0,4,0,41.089686,10253,0,0,20.015916,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,1,57,57,57,0,0,0.0,33,33,33,0,0,0,0,0,5,0,26.082693,9132,0,0,11.237107,0.0,0.0,0.0,0.0
11496,1,60,60,60,0,0,0.0,28,28,28,0,0,0,0,0,5,0,41.862065,9723,0,0,19.467072,0.0,0.0,0.0,0.0
11497,1,64,64,64,0,0,0.0,34,34,34,0,0,0,0,0,6,0,61.041030,10559,0,0,42.804220,0.0,0.0,0.0,0.0
11498,1,68,68,68,0,0,0.0,34,34,34,0,0,0,0,0,8,0,15.976726,10448,0,0,33.392447,0.0,0.0,0.0,0.0


## Then we fill values into columns that have a certain correlation between them:

A correlation between two or more columns is common in our dataset since most features are inherently related. All of them are derived from network packet traffic.<br>
For example, as the **flow duration increases**, the **packets per second** is likely to decrease. This occurs because each flow has an upper limit on duration, after which data collection stops and a new flow begins.<br>  
Similarly, the **Inter-Arrival Time (IAT)** of packets within a flow is influenced by the flow duration. Given these dependencies, <br>
the attack dataset should generate data for these columns collectively, ensuring that their inherent correlations are maintained.

### Correlation between 'SYN Flag Count' and 'Total Length of Fwd Packet':

In [17]:
# finding the correlation between the 'SYN Flag Count' column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'Total Length of Fwd Packet']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dos_samples[first_correlation[1]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'SYN Flag Count' and 'Total Length of Fwd Packet'
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond=None)[0]

scaling_factors = [(name, factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

('Total Length of Fwd Packet', np.float64(26.000000000000007))


After finding the scaling factors we can apply some randomness when generating values for the attack dataset in order to generate better data (without many duplications).<br>
We add randomness by creating a modified scaling factor, which introduces controlled variations in the generated values.<br>
This is done by selecting a small random delta (between 5% and 25% of the factor) and adding or subtracting it from the original scaling factor.<br>
As a result, the generated data maintains realistic correlations while avoiding exact duplicates.

In [18]:
# generate new data by scaling the original correlated column value using the updated factor.
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all generated scaling factors
        delta = random.uniform(factor * 0.05, factor * 0.25) 
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset.loc[index, col] = int(row['SYN Flag Count'] * updated_factor) 

In [19]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,68,68,68,0,0,278786.0,32,32,32,0,0,0,0,0,7,0,25.180855,9806,0,0,42.414232,0.0,0.0,0.0,0.0
1,1,64,64,64,0,0,186024.0,28,28,28,0,0,0,0,0,5,0,42.078845,8586,0,0,133.656006,0.0,0.0,0.0,0.0
2,1,66,66,66,0,0,257212.0,34,34,34,0,0,0,0,0,4,0,60.819446,10630,0,0,18.534921,0.0,0.0,0.0,0.0
3,1,51,51,51,0,0,265422.0,33,33,33,0,0,0,0,0,5,0,48.427463,9355,0,0,33.887945,0.0,0.0,0.0,0.0
4,1,55,55,55,0,0,223235.0,33,33,33,0,0,0,0,0,4,0,41.089686,10253,0,0,20.015916,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,1,57,57,57,0,0,267169.0,33,33,33,0,0,0,0,0,5,0,26.082693,9132,0,0,11.237107,0.0,0.0,0.0,0.0
11496,1,60,60,60,0,0,218498.0,28,28,28,0,0,0,0,0,5,0,41.862065,9723,0,0,19.467072,0.0,0.0,0.0,0.0
11497,1,64,64,64,0,0,326769.0,34,34,34,0,0,0,0,0,6,0,61.041030,10559,0,0,42.804220,0.0,0.0,0.0,0.0
11498,1,68,68,68,0,0,285861.0,34,34,34,0,0,0,0,0,8,0,15.976726,10448,0,0,33.392447,0.0,0.0,0.0,0.0


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [20]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dos_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = dos_samples[second_correlation[1:]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name, factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(dos_samples['Flow Duration'].values, dos_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

('Packets Per Second', np.float64(1.8396896437579322))
('IAT Max', np.float64(0.998429144907052))
('IAT Mean', np.float64(0.00015181064048409737))
('IAT Std', np.float64(0.01191782011351499))


np.float64(8585.611111111111)

And again here after finding the scaling factors we add some randomness and generate the data

In [21]:
# calculate a random small delta of the factor for some randomness
for index, row in dos_dataset.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr * 0.1, duration_to_packets_corr * 0.15)
            updatedFactor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = updatedFactor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.1, factor * 0.35)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            elif col == 'IAT Max':
                delta = random.uniform(factor * 0.1, factor * 0.225)
                updatedFactor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.05, factor * 0.2)
                updatedFactor = factor + random.choice([-1, 1]) * delta
            dos_dataset.loc[index, col] = row['Flow Duration'] * updatedFactor

In [22]:
dos_dataset

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,68,68,68,0,0,278786.0,32,32,32,0,0,0,0,0,7,0,25.180855,9806,0,0,42.414232,174.747771,33.218450,0.005404,0.350052
1,1,64,64,64,0,0,186024.0,28,28,28,0,0,0,0,0,5,0,42.078845,8586,0,0,133.656006,72.288032,115.226492,0.021633,1.293424
2,1,66,66,66,0,0,257212.0,34,34,34,0,0,0,0,0,4,0,60.819446,10630,0,0,18.534921,513.860798,15.833864,0.003083,0.271149
3,1,51,51,51,0,0,265422.0,33,33,33,0,0,0,0,0,5,0,48.427463,9355,0,0,33.887945,288.055745,30.381480,0.004653,0.471195
4,1,55,55,55,0,0,223235.0,33,33,33,0,0,0,0,0,4,0,41.089686,10253,0,0,20.015916,492.783320,17.904271,0.003609,0.272557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,1,57,57,57,0,0,267169.0,33,33,33,0,0,0,0,0,5,0,26.082693,9132,0,0,11.237107,877.879843,13.551327,0.001458,0.162407
11496,1,60,60,60,0,0,218498.0,28,28,28,0,0,0,0,0,5,0,41.862065,9723,0,0,19.467072,375.377128,15.950461,0.003321,0.193885
11497,1,64,64,64,0,0,326769.0,34,34,34,0,0,0,0,0,6,0,61.041030,10559,0,0,42.804220,173.497203,50.890010,0.006050,0.411937
11498,1,68,68,68,0,0,285861.0,34,34,34,0,0,0,0,0,8,0,15.976726,10448,0,0,33.392447,225.217679,27.604182,0.004236,0.483565


## Adding the Label column:

In [23]:
# adding a label to the dataset
dos_dataset['Label'] = ATTACK_NAME

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [24]:
dos_samples.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,18.0,18.0,18.0,18.0,18.0,18.0,18.000000,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.000000,18.000000,18.0,18.0,18.000000,18.000000,18.000000,18.000000,18.000000
mean,1.0,60.0,60.0,60.0,0.0,0.0,223225.888889,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,58.869588,8585.611111,0.0,0.0,50.730569,1261.620636,50.075167,0.007958,0.609617
std,0.0,0.0,0.0,0.0,0.0,0.0,67405.300285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,98.729675,2592.511549,0.0,0.0,47.078915,1854.329691,47.670273,0.009044,0.595760
min,1.0,60.0,60.0,60.0,0.0,0.0,70460.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,26.008564,2710.000000,0.0,0.0,1.574094,30.279728,1.323679,0.000157,0.014803
25%,1.0,60.0,60.0,60.0,0.0,0.0,240935.500000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,26.015761,9266.750000,0.0,0.0,5.008885,97.571008,1.748020,0.000501,0.028616
50%,1.0,60.0,60.0,60.0,0.0,0.0,257478.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,26.061350,9903.000000,0.0,0.0,44.862277,212.807671,44.743005,0.004704,0.458022
75%,1.0,60.0,60.0,60.0,0.0,0.0,259922.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,52.472023,9997.000000,0.0,0.0,75.907850,1997.661992,75.807678,0.010250,1.020746
max,1.0,60.0,60.0,60.0,0.0,0.0,260000.000000,26.0,26.0,26.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,449.826990,10000.000000,0.0,0.0,161.398189,6352.861489,161.285302,0.033034,1.999864


In [25]:
dos_dataset.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,11500.0,11500.000000,11500.000000,11500.000000,11500.0,11500.0,11500.000000,11500.000000,11500.000000,11500.000000,11500.0,11500.0,11500.0,11500.0,11500.0,11500.000000,11500.0,11500.000000,11500.000000,11500.0,11500.0,11500.000000,11500.000000,11500.000000,11500.000000,11500.000000
mean,1.0,61.003043,61.003043,61.003043,0.0,0.0,239086.300000,27.952174,27.952174,27.952174,0.0,0.0,0.0,0.0,0.0,5.989565,0.0,65.349569,9190.951478,0.0,0.0,39.808990,546.618466,39.766469,0.006047,0.473273
std,0.0,6.044742,6.044742,6.044742,0.0,0.0,51555.335802,3.735816,3.735816,3.735816,0.0,0.0,0.0,0.0,0.0,1.403682,0.0,80.742921,1302.016748,0.0,0.0,39.641469,720.617674,40.881837,0.006124,0.497173
min,1.0,51.000000,51.000000,51.000000,0.0,0.0,48364.000000,22.000000,22.000000,22.000000,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,13.768462,2305.000000,0.0,0.0,1.433927,41.464310,1.222179,0.000183,0.011898
25%,1.0,56.000000,56.000000,56.000000,0.0,0.0,203165.500000,25.000000,25.000000,25.000000,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,30.057758,8678.000000,0.0,0.0,15.258234,198.553311,14.821123,0.002262,0.171441
50%,1.0,61.000000,61.000000,61.000000,0.0,0.0,238758.000000,28.000000,28.000000,28.000000,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,45.726004,9341.000000,0.0,0.0,28.496156,301.679109,27.629229,0.004239,0.318595
75%,1.0,66.000000,66.000000,66.000000,0.0,0.0,279157.750000,31.000000,31.000000,31.000000,0.0,0.0,0.0,0.0,0.0,7.000000,0.0,62.260517,10021.000000,0.0,0.0,41.870214,557.948863,43.263276,0.006426,0.541872
max,1.0,71.000000,71.000000,71.000000,0.0,0.0,380277.000000,34.000000,34.000000,34.000000,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,494.792263,12095.000000,0.0,0.0,177.460782,6810.879964,215.207906,0.032027,2.798151


---

## Load the second sample dataset:

The following code will create another attack dataset, this time based on a different sample dataset, the code in this section<br> 
will be mostly the same as it was up until this point in the notebook, there for we will not repeat the same explanations here.<br>

In [26]:
NUM_OF_ROWS = 3500 #adjust the number of rows for the second dataset because samples like these almost never happen, thus should be less prominent in the final attack dataset

## Load the second sample dataset:

In [27]:
# import the attack sample dataset
dos_samples = pd.read_csv('dos_hping_samples_2.csv')
print(f'Dataset Shape: {dos_samples.shape}')
dos_samples

Dataset Shape: (8, 26)


,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,251472,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9672,0,0,0.113266,85391.915937,0.002451,0.000012,0.000077
1,1,60.0,60,60,0.0,0.0,259818,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9993,0,0,0.125600,79562.038842,0.013888,0.000013,0.000176
2,1,60.0,60,60,0.0,0.0,259558,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9983,0,0,0.122131,81740.025636,0.001796,0.000012,0.000070
3,1,60.0,60,60,0.0,0.0,250952,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9652,0,0,0.155204,62189.191252,0.002296,0.000016,0.000083
4,1,60.0,60,60,0.0,0.0,259844,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9994,0,0,0.150346,66473.316835,0.005427,0.000015,0.000100
5,1,60.0,60,60,0.0,0.0,258440,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9940,0,0,0.146744,67737.005470,0.001699,0.000015,0.000080
6,1,60.0,60,60,0.0,0.0,253734,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9759,0,0,0.133626,73032.202973,0.003685,0.000014,0.000085
7,1,60.0,60,60,0.0,0.0,259740,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9990,0,0,0.150142,66537.031827,0.013583,0.000015,0.000234


### Find an approximate minimum and maximum values of each column:

In [28]:
# find an approximate minimum and maximum values of each column and save that data into a dictionary
min_max_dict = {col: (dos_samples[col].min() * 0.85, dos_samples[col].max() * 1.1) for col in columns_to_gather}
min_max_dict['Number of Ports'] = (1, 1)

# print the min max dictionary
for col, (min_val, max_val) in min_max_dict.items():
    print(f'{col:<30} | Min: {min_val:.2f} | Max: {max_val:.2f}')

Number of Ports                | Min: 1.00 | Max: 1.00
Average Packet Length          | Min: 51.00 | Max: 66.00
Packet Length Min              | Min: 51.00 | Max: 66.00
Packet Length Max              | Min: 51.00 | Max: 66.00
Total Length of Fwd Packet     | Min: 213309.20 | Max: 285828.40
Fwd Packet Length Max          | Min: 22.10 | Max: 28.60
Fwd Packet Length Mean         | Min: 22.10 | Max: 28.60
Fwd Packet Length Min          | Min: 22.10 | Max: 28.60
Fwd Segment Size Avg           | Min: 5.10 | Max: 6.60
Subflow Fwd Bytes              | Min: 0.00 | Max: 0.00
SYN Flag Count                 | Min: 8204.20 | Max: 10993.40
Flow Duration                  | Min: 0.10 | Max: 0.17
Packets Per Second             | Min: 52860.81 | Max: 93931.11
IAT Max                        | Min: 0.00 | Max: 0.02
IAT Mean                       | Min: 0.00 | Max: 0.00
IAT Std                        | Min: 0.00 | Max: 0.00


### Create the base attack dataset (full of zeros):

In [29]:
# creating an empty dataframe before adding values to it
dos_dataset2 = pd.DataFrame(np.zeros((NUM_OF_ROWS, len(dos_samples.columns))), columns = dos_samples.columns)
dos_dataset2.head(3)

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Find the columns with constant zero values based on samples:

In [30]:
# adding zeros to all columns that should not have any values
zero_columns = [col for col in dos_samples.columns if col not in columns_to_gather]
for col in zero_columns:
    dos_dataset2[col] = int(0)
zero_columns

['Packet Length Std',
 'Packet Length Variance',
 'Fwd Packet Length Std',
 'Bwd Packet Length Max',
 'Bwd Packet Length Mean',
 'Bwd Packet Length Min',
 'Bwd Packet Length Std',
 'Bwd Segment Size Avg',
 'ACK Flag Count',
 'RST Flag Count']

---

## Filling in values based on collected samples:

### Firstly we insert data into columns that have the exact same values:

In [31]:
same_value = ['Average Packet Length', 'Packet Length Min', 'Packet Length Max']
val = np.random.randint(min_max_dict[same_value[0]][0], min_max_dict[same_value[0]][1]*1.1, NUM_OF_ROWS)

for col in same_value:
    dos_dataset2[col] = val

In [32]:
same_value2 = ['Fwd Packet Length Max', 'Fwd Packet Length Mean', 'Fwd Packet Length Min']
val2 = np.random.randint(min_max_dict[same_value2[0]][0], min_max_dict[same_value2[0]][1]*1.25, NUM_OF_ROWS)

for col in same_value2:
    dos_dataset2[col] = val2

### Then we insert data into columns that are independant of each other, based on the min max values:

In [33]:
dos_dataset2['Fwd Segment Size Avg'] = np.random.randint(min_max_dict['Fwd Segment Size Avg'][0]*0.9, min_max_dict['Fwd Segment Size Avg'][1]*1.5, NUM_OF_ROWS)
dos_dataset2['Flow Duration'] = np.random.uniform(min_max_dict['Flow Duration'][0]*0.95, min_max_dict['Flow Duration'][1]*1.05, NUM_OF_ROWS)
dos_dataset2['Number of Ports'] = np.full(shape = NUM_OF_ROWS, fill_value = 1, dtype = int)
dos_dataset2['Subflow Fwd Bytes'] = np.full(shape = NUM_OF_ROWS, fill_value = 0, dtype = int)
dos_dataset2['SYN Flag Count'] = np.random.randint(min_max_dict['SYN Flag Count'][0]*0.9, min_max_dict['SYN Flag Count'][1]*1.1, NUM_OF_ROWS)

## Then we fill values into columns that have a certain correlation between them:

### Correlation between 'SYN Flag Count' and 'Total Length of Fwd Packet':

In [34]:
# finding the correlation between the 'SYN Flag Count' column to the rest of the columns in order to create new data
first_correlation = ['SYN Flag Count', 'Total Length of Fwd Packet']
independent_col = dos_samples[first_correlation[0]].values.reshape(-1, 1) #column 'SYN Flag Count'
dependent_cols = dos_samples[first_correlation[1]].values 

# using least squares regression to find scaling factors that best approximate the relationship between 'SYN Flag Count' and 'Total Length of Fwd Packet'
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name,factor) for name, factor in zip(first_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)
    
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dos_dataset2.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'SYN Flag Count'
        delta = random.uniform(factor * 0.05, factor * 0.25) 
        updated_factor = factor + random.choice([-1, 1]) * delta
        dos_dataset2.loc[index, col] = int(row['SYN Flag Count'] * updated_factor)

('Total Length of Fwd Packet', np.float64(26.000000000000007))


### Correlation between 'Flow Duration' and all of the following: 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std':

In [35]:
# finding the correlation between the 'Flow Duration' column to the rest of the columns in order to create new data
second_correlation = ['Flow Duration', 'Packets Per Second', 'IAT Max', 'IAT Mean', 'IAT Std']
independent_col = dos_samples[second_correlation[0]].values.reshape(-1, 1) #column 'Flow Duration'
dependent_cols = dos_samples[second_correlation[1:]].values

# using least squares regression to find scaling factors that best approximate the relationship between 'Flow Duration' and the rest of the columns in second_correlation
scaling_factors = np.linalg.lstsq(independent_col, dependent_cols, rcond = None)[0]

scaling_factors = [(name,factor) for name, factor in zip(second_correlation[1:], scaling_factors.flatten())]
for val in scaling_factors:
    print(val)

# calculate the average correlation between flow duration and packets per second by multiplying their corresponding values from both columns and then calculate the average.
duration_to_packets_corr = [x * y for x, y in zip(dos_samples['Flow Duration'].values, dos_samples['Packets Per Second'].values)]
duration_to_packets_corr = np.mean(duration_to_packets_corr)
duration_to_packets_corr

('Packets Per Second', np.float64(519129.6051044216))
('IAT Max', np.float64(0.040715922209246726))
('IAT Mean', np.float64(0.00010129001644309197))
('IAT Std', np.float64(0.000825544773212792))


np.float64(9872.875)

In [36]:
# adding the rest of the attack feature values to the dataset at random based on the smaple data
for index, row in dos_dataset2.iterrows():
    for col, factor in scaling_factors: #iterating over all rows we need to add values to except 'Flow Duration'
        if col == 'Packets Per Second':
            delta = random.uniform(duration_to_packets_corr*0.025, duration_to_packets_corr * 0.075)
            updated_factor = duration_to_packets_corr + random.choice([-1, 1]) * delta
            dos_dataset2.loc[index, col] = updated_factor / row['Flow Duration']
        else:
            if col == 'IAT Std':
                delta = random.uniform(factor * 0.1, factor * 0.35)
                updated_factor = factor + random.choice([-1, 1]) * delta  
            elif col == 'IAT Max':
                delta = random.uniform(factor * 0.15, factor * 0.7)
                updated_factor = factor + random.choice([-1, 1]) * delta  
            else:
                delta = random.uniform(factor * 0.05, factor * 0.2) 
                updated_factor = factor + random.choice([-1, 1]) * delta
            dos_dataset2.loc[index, col] = row['Flow Duration'] * updated_factor

---

## Validate that the generated data looks valid by comparing the samples with the generated dataset:

In [37]:
dos_samples

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
0,1,60.0,60,60,0.0,0.0,251472,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9672,0,0,0.113266,85391.915937,0.002451,0.000012,0.000077
1,1,60.0,60,60,0.0,0.0,259818,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9993,0,0,0.125600,79562.038842,0.013888,0.000013,0.000176
2,1,60.0,60,60,0.0,0.0,259558,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9983,0,0,0.122131,81740.025636,0.001796,0.000012,0.000070
3,1,60.0,60,60,0.0,0.0,250952,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9652,0,0,0.155204,62189.191252,0.002296,0.000016,0.000083
4,1,60.0,60,60,0.0,0.0,259844,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9994,0,0,0.150346,66473.316835,0.005427,0.000015,0.000100
5,1,60.0,60,60,0.0,0.0,258440,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9940,0,0,0.146744,67737.005470,0.001699,0.000015,0.000080
6,1,60.0,60,60,0.0,0.0,253734,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9759,0,0,0.133626,73032.202973,0.003685,0.000014,0.000085
7,1,60.0,60,60,0.0,0.0,259740,26,26.0,26,0.0,0,0.0,0,0.0,6.0,0.0,0.0,9990,0,0,0.150142,66537.031827,0.013583,0.000015,0.000234


In [38]:
dos_dataset2.describe()

,Number of Ports,Average Packet Length,Packet Length Min,Packet Length Max,Packet Length Std,Packet Length Variance,Total Length of Fwd Packet,Fwd Packet Length Max,Fwd Packet Length Mean,Fwd Packet Length Min,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Mean,Bwd Packet Length Min,Bwd Packet Length Std,Fwd Segment Size Avg,Bwd Segment Size Avg,Subflow Fwd Bytes,SYN Flag Count,ACK Flag Count,RST Flag Count,Flow Duration,Packets Per Second,IAT Max,IAT Mean,IAT Std
count,3500.0,3500.000000,3500.000000,3500.000000,3500.0,3500.0,3500.000000,3500.000000,3500.000000,3500.000000,3500.0,3500.0,3500.0,3500.0,3500.0,3500.000000,3500.0,3500.0,3500.000000,3500.0,3500.0,3500.000000,3500.000000,3500.000000,3500.000000,3500.000000
mean,1.0,61.060857,61.060857,61.060857,0.0,0.0,253299.638000,28.003429,28.003429,28.003429,0.0,0.0,0.0,0.0,0.0,5.988857,0.0,0.0,9752.170286,0.0,0.0,0.135393,75710.268450,0.005540,0.000014,0.000112
std,0.0,6.071218,6.071218,6.071218,0.0,0.0,54036.992368,3.722665,3.722665,3.722665,0.0,0.0,0.0,0.0,0.0,1.396988,0.0,0.0,1370.480411,0.0,0.0,0.025438,15332.024843,0.002768,0.000003,0.000034
min,1.0,51.000000,51.000000,51.000000,0.0,0.0,146370.000000,22.000000,22.000000,22.000000,0.0,0.0,0.0,0.0,0.0,4.000000,0.0,0.0,7384.000000,0.0,0.0,0.091494,51017.287185,0.001178,0.000008,0.000051
25%,1.0,56.000000,56.000000,56.000000,0.0,0.0,212593.750000,25.000000,25.000000,25.000000,0.0,0.0,0.0,0.0,0.0,5.000000,0.0,0.0,8593.000000,0.0,0.0,0.113456,62899.022098,0.003107,0.000011,0.000086
50%,1.0,61.000000,61.000000,61.000000,0.0,0.0,248868.500000,28.000000,28.000000,28.000000,0.0,0.0,0.0,0.0,0.0,6.000000,0.0,0.0,9756.500000,0.0,0.0,0.134971,72916.003803,0.005203,0.000013,0.000106
75%,1.0,66.000000,66.000000,66.000000,0.0,0.0,290560.750000,31.000000,31.000000,31.000000,0.0,0.0,0.0,0.0,0.0,7.000000,0.0,0.0,10935.500000,0.0,0.0,0.157218,87228.610776,0.007813,0.000016,0.000135
max,1.0,71.000000,71.000000,71.000000,0.0,0.0,389189.000000,34.000000,34.000000,34.000000,0.0,0.0,0.0,0.0,0.0,8.000000,0.0,0.0,12091.000000,0.0,0.0,0.179160,115349.134000,0.012268,0.000022,0.000198


## Adding the Label column:

In [39]:
dos_dataset2['Label'] = ATTACK_NAME

---

## At the end we merge both attack datasets into one final attack dataset for DoS Hping and save the dataset as a CSV file

In [40]:
# merging and shuffling the rows in the final dataset of the DoS Hping attack
merged_dos_dataset = pd.concat([dos_dataset, dos_dataset2], axis=0)
merged_dos_dataset = merged_dos_dataset.sample(frac=1, random_state=42).reset_index(drop=True)
print(f'Attack Dataset Shape: {merged_dos_dataset.shape}')

Attack Dataset Shape: (15000, 27)


In [ ]:
# save the dataset
merged_dos_dataset.to_csv('dos_hping_dataset.csv', index=False)

---